# Préparation de l'espace de travail

## Importations

In [ ]:
!pip install openpyxl cartiflette mapclassify

import openpyxl 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
from cartiflette import carti_download
import mapclassify
import plotly.express as px

## Lecture des bases de données

### Base de la démographie des médecins : medecin

In [ ]:
# Lecture du Excel medecins 
df_medecins_effectif_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=1)
df_medecins_age_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=2)
df_medecins_densite_complet = pd.read_excel("Bases de données/demographie_medecins.xlsx", sheet_name=3)

In [ ]:
# Traitement de la feuille des effectifs : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_effectif = df_medecins_effectif_complet[(df_medecins_effectif_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_effectif_complet['sexe'] == '0-Ensemble')
    & (df_medecins_effectif_complet['specialites'] == '00-Ensemble')
    ]

df_medecins_effectif.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_medecins_effectif.reset_index(drop = True, inplace = True)

df_medecins_effectif

In [ ]:
# Traitement de la feuille des âges moyens : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_age = df_medecins_age_complet[(df_medecins_age_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_age_complet['sexe'] == '0-Ensemble')
    & (df_medecins_age_complet['specialites'] == '00-Ensemble')]

df_medecins_age.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_medecins_age.reset_index(drop = True, inplace = True)

df_medecins_age

In [ ]:
# Traitement de la feuille des densités : on ne conserve que les médecins généralistes et on enlève les variables pas pertinentes ou redondantes
df_medecins_densite = df_medecins_densite_complet[(df_medecins_densite_complet['specialites_agregees'] == '1-Médecine générale') 
    & (df_medecins_densite_complet['sexe'] == '0-Ensemble')
    & (df_medecins_densite_complet['specialites'] == '00-Ensemble')]

df_medecins_densite.drop(axis = 1, columns = ['sexe', 'specialites', 'specialites_agregees'], inplace = True)
df_medecins_densite.reset_index(drop = True, inplace = True)

df_medecins_densite

### Base de la prévalence de différentes maladies par groupe de population : santePublique

In [ ]:
# Lecture du csv santePublique
import zipfile

with zipfile.ZipFile("Bases de données/santePublique.zip", 'r') as zip_ref:
    zip_ref.extractall("Bases de données")

df_sante_publique = pd.read_csv('Bases de données/santePublique.csv', sep=";")

### Base de l'indicateur APL (Accessibilité Potentielle Localisée) pour les médecins généralistes 

Cet indicateur est une mesure de l'accessibilité aux médecins libéraux, qui tient compte du niveau d'activité des médecins (offre) et du niveau de recours de la population (demande). Cet indicateur est construit au niveau communal mais prend en compte l'offre et la demande des communes voisines, dans un certain périmètre.
Pour son calcul, on définit une zone de recours et une zone de patientèle. Les médecins sont comptés en ETP (équivalent temps plein), afin de prendre en compte leur activité annuelle.
L'APL nous donne finalement le nombre d'ETP des médecins généralistes libéraux pour 100 000 habitants.
Les données de 2015 à 2021 n'utilisent pas exactement la même méthode que de 2022 à 2023, donc ces données ne sont pas parfaitement comparables.

In [ ]:
# Lecture du Excel APL_2022_2023 : tableau de la pondération de la population
# (consommation moyenne en soins de la tranche d'âge rapportée à la consommation moyenne de la population)
df_pond_population_2022_2023 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=0)[21:40]
df_pond_population_2022_2023.reset_index(drop=True, inplace=True)

# Renommage des colonnes
df_pond_population_2022_2023.columns = ["Tranche d'âge", "Poids de la tranche d'âge en 2022", "Poids de la tranche d'âge en 2023"]

# Lecture du Excel APL_2015_2022 : tableau de pondération de la population (on exclut l'année 2022 en supposant que les données de APL_2022_2023 sont plus pertinentes)
df_pond_population_2015_2021 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=0)[21:40]
df_pond_population_2015_2021.reset_index(drop=True, inplace=True)

# Renommage des colonnes
df_pond_population_2015_2021.columns = ["Tranche d'âge", "Poids de la tranche d'âge en 2015", "Poids de la tranche d'âge en 2016", "Poids de la tranche d'âge en 2017", 
    "Poids de la tranche d'âge en 2018", "Poids de la tranche d'âge en 2019", "Poids de la tranche d'âge en 2021", "Poids de la tranche d'âge en 2022 bis"]
df_pond_population_2015_2021.drop("Poids de la tranche d'âge en 2022 bis", axis=1, inplace=True)

# Merging des deux tables sur la clé de la tranche d'âge
df_pond_population = df_pond_population_2015_2021.merge(right=df_pond_population_2022_2023, how="left", on=["Tranche d'âge"])

df_pond_population

In [ ]:
# Lecture des deux feuilles de la table APL_2022_2023
df_APL_2022 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=1)[8:]
df_APL_2023 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=2)[8:]

# Nettoyage et renommage des colonnes
bases = [df_APL_2022, df_APL_2023]
annee = 2022

for base in bases : 
    base.drop(8,inplace=True)
    base.reset_index(drop=True, inplace=True)
    base.columns = ["Code commune INSEE", "Commune", f"APL_{annee}", f"APL_{annee}_moins_65", f"APL_{annee}_moins_62", f"APL_{annee}_moins_60", f"population_standard_{annee-2}", f"population_totale_{annee-2}"]
    annee += 1

# Lecture des feuilles de la table APL_2015_2022 (on exclut l'année 2022 en supposant que les données de APL_2022_2023 sont plus pertinentes)
df_APL_2015 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=1)[8:]
df_APL_2016 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=2)[8:]
df_APL_2017 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=3)[8:]
df_APL_2018 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=4)[8:]
df_APL_2019 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=5)[8:]
df_APL_2021 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=6)[8:]

# Nettoyage et renommage des colonnes
bases = [df_APL_2015,df_APL_2016,df_APL_2017,df_APL_2018,df_APL_2019,df_APL_2021]
annee = 2015

for base in bases : 
    base.drop(8,inplace=True)
    base.reset_index(drop=True, inplace=True)
    base.columns = ["Code commune INSEE", "Commune", f"APL_{annee}", f"APL_{annee}_moins_65", f"population_standard_{annee-2}", f"population_totale_{annee-2}"]
    annee += 1
    if annee == 2020 : 
        annee += 1

bases.append(df_APL_2022)
df_APL = df_APL_2023
for base in bases : 
    df_APL = df_APL.merge(base, how='left', on=["Code commune INSEE", "Commune"])

df_APL['departement'] = df_APL['Code commune INSEE'].astype(str).str[:2]

In [ ]:
# On crée l'indicateur  APL par département pour l'année 2023
df_APL["APL_2023"] = pd.to_numeric(df_APL["APL_2023"], errors="coerce")

df_APL['APL_2023_pond'] = df_APL['APL_2023'] * df_APL['population_standard_2021']
df_APL["APL_dep_2023"] = df_APL.groupby("departement")['APL_2023_pond'].transform("mean")

df_APL["APL_dep_2023"] = pd.to_numeric(df_APL["APL_dep_2023"], errors="coerce")

# On crée l'indicateur  APL par département pour les années 2015 à 2022, 2020 exclu
annees = [2015, 2016, 2017, 2018, 2019, 2021, 2022]

for annee in annees : 
    df_APL[f"APL_{annee}"] = pd.to_numeric(df_APL[f"APL_{annee}"], errors="coerce")
    df_APL[f'APL_{annee}_pond'] = df_APL[f'APL_{annee}'] * df_APL[f'population_standard_{annee - 2}']

    df_APL[f"APL_dep_{annee}"] = df_APL.groupby("departement")[f'APL_{annee}_pond'].transform("mean")
    df_APL[f"APL_dep_{annee}"] = pd.to_numeric(df_APL[f"APL_dep_{annee}"], errors="coerce")

### Base de la patientèle des médecins 

In [ ]:
df_patientele = pd.read_csv("Bases de données/Données_Patientele_Departementale.csv", sep=";")

# Enlever les caractères cachés dans le nom des colonnes (provoquaient des bugs dans la suite du code)
df_patientele.columns = df_patientele.columns.str.replace('\ufeff', '').str.strip()

df_patientele["nombre_patients_uniques"] = pd.to_numeric(df_patientele["nombre_patients_uniques"], errors="coerce")
df_patientele['profession_sante'].unique()

### Base d'indicateurs démographiques au niveau communal

In [ ]:
df_pop_communes = pd.read_csv("Bases de données/Indicateurs_communale.csv", sep = ';')
df_pop_communes.columns = df_pop_communes.loc[1]
df_pop_communes.drop([0,1], inplace = True)

# Nettoyage : on remarque plus tard dans le code qu'il y a un problème d'espace avec la colonne Code
df_pop_communes['Code'] = df_pop_communes['Code'].astype(str).str.strip().str.zfill(5)

# Nettoyage : conversion des valeurs NA et NS en np.nan 
df_pop_communes.replace(["N/A - résultat non disponible","N/A - secret statistique", "N/A - résultat non disponibleN/A - résultat non disponibleN/A - résultat non disponible", "N/A - division par 0"],
    np.nan, inplace = True)

# Conversion des colonnes en numérique
for col in df_pop_communes.columns : 
    if col not in ['Code', 'Libellé']:
        df_pop_communes[col] = pd.to_numeric(df_pop_communes[col], errors="coerce")

In [ ]:
# Construction de l'indicateur de densité de médecins généralistes par commune (nombre de médecins pour 100 000 habitans)
df_pop_communes['Densité médecins généralistes 2024'] = df_pop_communes['Médecin généraliste (en nombre) 2024']/df_pop_communes['Population municipale 2023']*100000

# On ajoute la variable du département
def extraction(x):
    if x[:2] in ['2A', '2B'] : 
        return x[:2]
    if int(x[:2]) < 96 : 
        return x[:2]
    return x[:3]

df_pop_communes['departement'] = df_pop_communes['Code'].astype(str).apply(extraction)

df_pop_communes

# Premières visualisations des bases

## Base de la démographie des médecins

### Diagramme circulaire du type d'exercice des médecins généralistes

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']!='0-Ensemble') 
    & (df_medecins_effectif['tranche_age']=='00-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")] 

df = df_cond.copy()

labels = df['exercice'].unique()
sizes = df['effectif_2023'].unique()

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels)

### Evolution du nombre de généralistes en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') 
    & (df_medecins_effectif['tranche_age']=='00-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble')
    & (df_medecins_effectif['departement'] == '000-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 

# Définition des ordonnées : effectifs des années 2012 à 2025 
effectifs = df[[f"effectif_{a}" for a in annees]].values.flatten() 
plt.plot(annees, effectifs, marker = "o") 

# Affichage du graphique 
plt.ylabel("Nombre de médecins généralistes") 
plt.title("Évolution du nombre de médecins généralistes en France") 

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes en France est passé d'environ 101 500 en 2012 à 100 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution du nombre de généralistes par région

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') & (df_medecins_effectif['tranche_age']=='00-Ensemble') & (df_medecins_effectif['region'] != '00-Ensemble')]
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026))

# Définition des ordonnées : effectifs par région des années 2012 à 2025 
for region in df['region'].unique() :
    df_region = df[(df['region'] == region) & (df['departement'] == '000-Ensemble')]
    effectifs = df_region[[f"effectif_{a}" for a in annees]].values.flatten()
    plt.plot(annees, effectifs, label = region, marker = "o")

# Affichage du graphique 
plt.xlabel("Année")
plt.ylabel("Nombre de médecins généralistes")
plt.title(f"Évolution du nombre de médecins généralistes par région")

# Légende
plt.legend(title="Région", bbox_to_anchor=(1.05, 1), loc="upper left")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes par région en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes en Île de France est passé d'environ 18 500 en 2012 à 16 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution de l'âge moyen des généralistes en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_age[(df_medecins_age['exercice']=='0-Ensemble') 
    & (df_medecins_age['region'] == '00-Ensemble') 
    & (df_medecins_age['territoire'] == "0-France entière")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 


# Définition des ordonnées : effectifs des années 2012 à 2025 
effectifs = df[[f"am_{a}" for a in annees]].values.flatten() 
plt.plot(annees, effectifs, marker = "o") 

# Affichage du graphique plt.xlabel("Année") 
plt.ylabel("Âge moyen des médecins généralistes") 
plt.title("Évolution de l'âge moyen des médecins généralistes en France") 
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : L'âge moyen des médecins généralistes en France est passé de 51,1 ans en 2012 à 50,4 ans en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution du nombre de médecins généralistes par tranche d'âge en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")
    & (df_medecins_effectif['tranche_age'] != "00-Ensemble")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 

# Définition des ordonnées : effectifs par tranche des années 2012 à 2025 
for tranche in df['tranche_age'].unique() : 
    df_age = df[df['tranche_age'] == tranche]
    effectifs = df_age[[f"effectif_{a}" for a in annees]].values.flatten() 
    plt.plot(annees, effectifs, label = tranche, marker = "o") 

# Affichage du graphique 
plt.ylabel("Nombre de médecins généralistes") 
plt.title("Évolution du nombre de médecins généralistes par tranche d'âge en France") 

# Légende
plt.legend(title="Tranche d'âge", bbox_to_anchor=(1.05, 1), loc="upper left")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : Le nombre de médecins généralistes entre 55 et 59 ans en France est passé d'environ 19 000 en 2012 à 10 000 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Evolution de la proportion des médecins généralistes par tranche d'âge en France

In [ ]:
# Définition de la base de données réduite 
df_cond = df_medecins_effectif[(df_medecins_effectif['exercice']=='0-Ensemble') 
    & (df_medecins_effectif['region'] == '00-Ensemble') 
    & (df_medecins_effectif['territoire'] == "0-France entière")] 
df = df_cond.copy()

# Définition des abscisses : années 2012 à 2025 
annees = list(range(2012,2026)) 

# Définition de nouvelles variables proportion par année
for annee in annees:
    total = df.loc[df['tranche_age'] == '00-Ensemble', f'effectif_{annee}'].values[0]
    df[f'proportion_{annee}'] = df[f'effectif_{annee}'] / total

# Définition des nuances de couleurs
cmap = cm.get_cmap("OrRd")
colors = cmap(np.linspace(0, 1, 12))
i = 0

# Définition des ordonnées : proportion des tranches des années 2012 à 2025 
for tranche in df['tranche_age'].unique() : 
    if tranche != "00-Ensemble" :
        df_age = df[df['tranche_age'] == tranche]
        proportion = df_age[[f"proportion_{a}" for a in annees]].values.flatten()
        plt.plot(annees, proportion, label = tranche, marker = "o", color = colors[i]) 
        i += 1

# Affichage du graphique 
plt.ylabel("Proportion des médecins généralistes") 
plt.title("Évolution de la proportion de médecins généralistes par tranche d'âge en France") 

# Légende
plt.legend(title="Tranche d'âge", bbox_to_anchor=(1.05, 1), loc="upper left")

# Sous-texte
plt.text(0, -0.16, "Source : DREES (RPPS), La démographie des professionnels de santé depuis 2012.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.21, "Champ : Médecins généralistes en France, DROM inclus.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 
plt.text(0, -0.26, "Lecture : La proportion de médecins généralistes entre 30 et 34 ans en France est passée de 0.06 en 2012 à 0.125 en 2025.", ha="left", va="bottom", fontsize=9, transform=plt.gca().transAxes) 

plt.grid(True)
plt.show()

### Etude de la densité de médecin par département pour l'année 2023

In [ ]:
df = df_medecins_densite.copy()
df = df[(df['departement'] != '000-Ensemble') 
    & (df['tranche_age'] == '00-Ensemble')
    & (df['exercice'] == '0-Ensemble')]

df.reset_index(inplace = True, drop = True)

df['departement'] = df['departement'].astype(str).str[:3]

# Départements

departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

departements['INSEE_DEP'] = departements['INSEE_DEP'].str.zfill(3)

# Test d'affichage de la carte
departements.plot().axis('off')

In [ ]:
# On joint les deux bases
departements = departements.merge(df, left_on = "INSEE_DEP", right_on = "departement", how="left")
departements = departements.to_crs(2154)

departements

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="densite_2023",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
)

ax.axis("off")
ax.set_title("Densité de médecins par département en 2023", fontsize=14)

plt.show()

## Base de l'indicateur APL

### Etude de l'indicateur APL par commune/département pour l'année 2023

In [ ]:
df_APL['APL_2023'].dropna().describe()
df_APL['APL_dep_2023'].dropna().describe()

### Cartographie de l'indicateur APL par commune pour l'année 2023

In [ ]:
df = df_APL.copy()
departements = df_APL['departement'].unique()

# 1. Communes
communes = carti_download(
    values = departements,
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

# 2. Départements 
departements = communes.dissolve("INSEE_DEP")

communes = communes.merge(df, left_on = "INSEE_COG", right_on = "Code commune INSEE", how="left")
communes = communes.to_crs(2154)

In [ ]:
communes.plot()

In [ ]:
# Réalisation de la carte
fig, ax = plt.subplots(figsize=(10,10))

# On choisit le nombre de quantiles : 
n = 5

communes.plot(
    ax=ax,
    column="APL_2023",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
    scheme = "quantiles",
    k = n, 
    legend_kwds={"labels": [i for i in range(n)]},
    missing_kwds={"color": "lightgrey", "label": "Données manquantes"}
)

ax.axis("off")
ax.set_title("Indicateur APL par commune en 2023", fontsize=14)

q = mapclassify.Quantiles(communes['APL_2023'].dropna(), k=n)
mapping = {i: s for i, s in enumerate(q.get_legend_classes())}
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k, v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)
replace_legend_items(ax.get_legend(), mapping)

plt.show()

### Cartographie de l'indicateur APL par département pour l'année 2023

In [ ]:
# Départements

departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Test d'affichage de la carte
departements.plot().axis('off')

In [ ]:
# On crée une copie de la table
df = df_APL.copy()

# On refait une carte, cette fois-ci des départements
df.drop_duplicates('departement', inplace = True)
departements = departements.merge(df, left_on = "INSEE_DEP", right_on = "departement", how="left")
departements = departements.to_crs(2154)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="APL_dep_2023",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
    scheme = "quantiles",
    k = 10, 
    legend_kwds={"labels": [i for i in range(10)]},
    missing_kwds={"color": "lightgrey", "label": "Données manquantes"}
)

ax.axis("off")
ax.set_title("Indicateur APL par département en 2023", fontsize=14)

q10 = mapclassify.Quantiles(departements['APL_dep_2023'].dropna(), k=10)
mapping = {i: s for i, s in enumerate(q10.get_legend_classes())}
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k, v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)
replace_legend_items(ax.get_legend(), mapping)

plt.show()

In [ ]:
departements["APL_dep_2023_log"] = np.log1p(departements["APL_dep_2023"])

fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="APL_dep_2023_log",
    cmap="OrRd",               
    linewidth=0,
    legend=True,
    edgecolor="lightgrey", 
)

ax.axis("off")
ax.set_title("Indicateur APL logarithmique par département en 2023", fontsize=14)

plt.show()

## Base des indicateurs communaux

In [ ]:
df = df_pop_communes.copy()
departements = df_pop_communes['departement'].unique()

# On limite les départements à ceux de la France métropolitaine pour des soucis d'affichage
departements = departements[:96]

In [ ]:
# 1. Communes
communes = carti_download(
    values = departements,
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

communes = communes.merge(df, right_on = 'Code', left_on = 'INSEE_COM')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

communes.plot(
    ax=ax,
    column="Densité médecins généralistes 2024",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
)

ax.axis("off")
ax.set_title("Densité de médecins par commune en 2024", fontsize=14)

plt.show()

In [ ]:
# Réalisation de la carte
fig, ax = plt.subplots(figsize=(10,10))

# On choisit le nombre de quantiles : 
n = 5

communes.plot(
    ax=ax,
    column="Densité médecins généralistes 2024",
    cmap="OrRd",               
    linewidth=0,
    edgecolor="lightgrey",
    legend=True,
    scheme = "NaturalBreaks",
    k = n, 
    legend_kwds={"labels": [i for i in range(n)]},
    missing_kwds={"color": "lightgrey", "label": "Données manquantes"}
)

ax.axis("off")
ax.set_title("Densité de médecins généralistes par commune en 2024", fontsize=14)

q = mapclassify.NaturalBreaks(communes['Densité médecins généralistes 2024'].dropna(), k=n)
mapping = {i: s for i, s in enumerate(q.get_legend_classes())}
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k, v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)
replace_legend_items(ax.get_legend(), mapping)

plt.show()

## Base de la patientèle (nombre de patients uniques par médecin)

### Etude du nombre moyen de patients par médecin en 2017

In [ ]:
# On s'intéresse d'abord uniquement à l'année 2017
df = df_patientele.copy()
df = df[(df['annee'] == 2017) & (df['profession_sante'] == 'Ensemble des médecins généralistes')]

df['nombre_patients_uniques']

In [ ]:
# Départements
departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Test d'affichage de la carte
departements.plot().axis('off')
departements['INSEE_DEP'].unique()

In [ ]:
# On refait une carte, cette fois-ci des départements

df.drop_duplicates('departement', inplace = True)
departements = departements.merge(df, left_on = "INSEE_DEP", right_on = "departement", how="left")
departements = departements.to_crs(2154)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(
    ax=ax,
    column="nombre_patients_uniques",
    cmap="OrRd",               
    linewidth=0.05,
    legend=True,
    edgecolor="lightgrey", 
)

ax.axis("off")
ax.set_title("Nombre moyen de patiens par médecin en 2017", fontsize=14)

plt.show()

# Modélisation : corrélation entre APL et différentes variables

### Premier test uniquement sur l'année 2023

In [ ]:
# On réarrange les tables APL densités de médecins
df1 = df_APL.copy()
df1['departement'] = df1['departement'].str.zfill(3)
df1.drop_duplicates('departement', inplace = True)

df2 = df_medecins_densite.copy()
df2 = df2[(df2['departement'] != '000-Ensemble') 
    & (df2['tranche_age'] == '00-Ensemble')
    & (df2['exercice'] == '0-Ensemble')]
df2['departement'] = df2['departement'].astype(str).str[:3]

df = df1.merge(df2, on = 'departement')
df.reset_index(drop = True, inplace = True)

df['APL_dep_2023']

In [ ]:
sns.regplot(
    data = df,
    x = 'densite_2023',
    y = 'APL_dep_2023',
    fit_reg = True,
    scatter_kws={'alpha':0.5}
)

### On étend la régression aux années pour lesquelles on a des données : 2015 à 2023, 2020 exclu

In [ ]:
# On réarrange les tables APL densités de médecins
df1 = df_APL.copy()
df1['departement'] = df1['departement'].str.zfill(3)

df2 = df_medecins_densite.copy()
df2 = df2[(df2['departement'] != '000-Ensemble') 
    & (df2['tranche_age'] == '00-Ensemble')
    & (df2['exercice'] == '0-Ensemble')]
df2['departement'] = df2['departement'].astype(str).str[:3]

df1.drop_duplicates('departement', inplace = True)

In [ ]:
# On transforme les tables larges en tables longues de manière à pouvoir faire la régression en utilisant les données de toutes les années
annees = [2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]
APL = [f"APL_dep_{annee}" for annee in annees]
densite = [f"densite_{annee}" for annee in annees] 

# Pour l'APL
df1_long = df1.melt(
    id_vars = ['departement'], 
    value_vars = APL, 
    var_name = 'annee',
    value_name = 'APL'
)

df1_long['annee'] = df1_long['annee'].astype(str).str[8:]

# Pour la densité
df2_long = df2.melt(
    id_vars = ['departement'], 
    value_vars = densite, 
    var_name = 'annee',
    value_name = 'densite' 
)

df2_long['annee'] = df2_long['annee'].astype(str).str[8:]

# On joint les deux tables
df = pd.merge(df2_long, df1_long, on=['departement', 'annee'])
df['APL'] = np.log(df['APL'])

In [ ]:
# Régression utilisant les données toutes années confondues
sns.regplot(
    data = df,
    x = 'densite',
    y = 'APL',
    fit_reg = True,
    scatter_kws={'alpha':0.5},
    color="lightblue", line_kws=dict(color="orange")
)

### Carte interactive pour l'évolution de l'APL par département

In [ ]:
# On transforme la tables large en table longue 
df1 = df_APL.copy()
annees = [2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]
APL = [f"APL_dep_{annee}" for annee in annees]

df1_long = df1.melt(
    id_vars = ['departement'], 
    value_vars = APL, 
    var_name = 'annee',
    value_name = 'APL'
)

df1_long['annee'] = df1_long['annee'].astype(str).str[8:]
df1_long

In [ ]:
# Départements
departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Test d'affichage de la carte
departements.plot().axis('off')
departements['INSEE_DEP'].unique()

gdf_carte = departements.merge(df1_long, left_on='INSEE_DEP', right_on='departement')

# On trie impérativement par année pour l'animation
gdf_carte = gdf_carte.sort_values('annee')

# On réinitialise l'index pour que Plotly s'y retrouve
gdf_carte = gdf_carte.reset_index(drop=True)

In [ ]:
# Fais crash le serveur donc à voir

# fig = px.choropleth(
#     gdf_carte,
#     geojson=gdf_carte.__geo_interface__,
#     locations=gdf_carte.index, # On utilise le nouvel index
#     color='APL',
#     animation_frame='annee',
#     range_color=[0, 4], 
#     scope="europe"
# )
# 
# fig.update_geos(fitbounds="locations", visible=False)
# fig.show()

# Lasso

### Importations

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
import sklearn.metrics
from sklearn.linear_model import LinearRegression
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.linear_model import lasso_path
import seaborn as sns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.linear_model import lasso_path
from sklearn.linear_model import LassoCV
import numpy as np

### Corrélations entre les variables

In [ ]:
def plot_corr_heatmap(
    df: pd.DataFrame,
    drop_cols=None,
    column_labels: dict | None = None,
    decimals: int = 2,
    width: int = 600,
    height: int = 600,
    show_xlabels: bool = False
):
    """
    Trace une heatmap de corrélation (triangle inférieur) à partir d'un DataFrame.

    Paramètres
    ----------
    df : pd.DataFrame
        DataFrame d'entrée.
    drop_cols : list ou None
        Liste de colonnes à supprimer avant le calcul de la corrélation
        (ex: ['winner']).
    column_labels : dict ou None
        Dictionnaire pour renommer les colonnes (ex: column_labels).
    decimals : int
        Nombre de décimales pour l'arrondi avant corr().
    width, height : int
        Dimensions de la figure Plotly.
    show_xlabels : bool
        Afficher ou non les labels en abscisse.
    """
    data = df.copy()

    # 1. Colonnes à drop
    if drop_cols is not None:
        data = data.drop(columns=drop_cols)

    # 2. Arrondi + renommage éventuel
    if column_labels is not None:
        data = data.rename(columns=column_labels)
    data = data.round(decimals)

    # 3. Matrice de corrélation
    corr = data.corr()

    # 4. Masque triangle supérieur
    mask = np.triu(np.ones_like(corr, dtype=bool))
    corr_masked = corr.mask(mask)

    # 5. Heatmap Plotly
    fig = px.imshow(
        corr_masked.values,
        x=corr.columns,
        y=corr.columns,
        color_continuous_scale='RdBu_r',  # échelle inversée
        zmin=-1,
        zmax=1,
        text_auto=".2f"
    )

    # 6. Hover custom
    fig.update_traces(
        hovertemplate="Var 1: %{y}<br>Var 2: %{x}<br>Corr: %{z:.2f}<extra></extra>"
    )

    # 7. Layout
    fig.update_layout(
        coloraxis_showscale=False,
        xaxis=dict(
            showticklabels=show_xlabels,
            title=None,
            ticks=''
        ),
        yaxis=dict(
            showticklabels=show_xlabels,
            title=None,
            ticks=''
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(t=10, b=10, l=10, r=10),
        width=width,
        height=height
    )

    return fig


df = df_pop_communes.copy().drop(columns=["Code", "Libellé", "departement", "Densité médecins généralistes 2024"])
plot_corr_heatmap(df)

In [ ]:
def plot_corr_with_target(
    df: pd.DataFrame,
    target: str,
    drop_cols=None,
    decimals: int = 2,
    width: int = 500,
    height: int = 900
):
    """
    Trace une heatmap des corrélations entre une variable cible
    et les autres variables du DataFrame.
    """
    data = df.copy()

    if drop_cols is not None:
        data = data.drop(columns=drop_cols)

    data = data.round(decimals)

    # Corrélation avec la variable cible
    corr = data.corr()[[target]].sort_values(by=target)

    fig = px.imshow(
        corr.values,
        x=[target],
        y=corr.index,
        color_continuous_scale="RdBu_r",
        zmin=-1,
        zmax=1,
        text_auto=".2f"
    )

    fig.update_traces(
        hovertemplate="Variable: %{y}<br>Corr: %{z:.2f}<extra></extra>"
    )

    fig.update_layout(
        coloraxis_showscale=False,
        xaxis_title=None,
        yaxis_title=None,
        width=width,
        height=height,
        margin=dict(t=20, b=20, l=20, r=20)
    )

    return fig

df = df_pop_communes.copy().drop(columns=["Code", "Libellé", "departement", "Densité médecins généralistes 2024"])
plot_corr_with_target(df,"Médecin généraliste (en nombre) 2024")

In [ ]:
def extract_features_selected(lasso: Pipeline, preprocessing_step_name: str = 'preprocess') -> pd.Series:
    """
    Extracts selected features based on the coefficients obtained from Lasso regression.

    Parameters:
    - lasso (Pipeline): The scikit-learn pipeline containing a trained Lasso regression model.
    - preprocessing_step_name (str): The name of the preprocessing step in the pipeline. Default is 'preprocess'.

    Returns:
    - pd.Series: A Pandas Series containing selected features with non-zero coeffaicients.
    """
    # Check if lasso object is provided
    if not isinstance(lasso, Pipeline):
        raise ValueError("The provided lasso object is not a scikit-learn pipeline.")
    
    # Extract the final transformer from the pipeline
    lasso_model = lasso[-1]

    # Check if lasso_model is a Lasso regression model
    if not isinstance(lasso_model, Lasso):
        raise ValueError("The final step of the pipeline is not a Lasso regression model.")

    # Check if lasso model has 'coef_' attribute
    if not hasattr(lasso_model, 'coef_'):
        raise ValueError("The provided Lasso regression model does not have 'coef_' attribute. "
                         "Make sure it is a trained Lasso regression model.")

    # Get feature names from the preprocessing step
    features_preprocessing = lasso[preprocessing_step_name].get_feature_names_out()

    # Extract selected features based on non-zero coefficients
    features_selec = pd.Series(features_preprocessing[np.abs(lasso_model.coef_) > 0])

    return features_selec

### Implémentations de Lasso

In [ ]:
df = df_pop_communes.copy().drop(columns=["Code", "Libellé", "departement", "Densité médecins généralistes 2024"])

# Définition de X (variables explicatives) et y (cible)
X = df.drop(columns=["Médecin généraliste (en nombre) 2024"]) 
y = df["Médecin généraliste (en nombre) 2024"]

# Séparation de la base en une base d'entraînement et une base de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifier les lignes où la valeur du nombre de médecins est manquante
mask = y_train.isna()

# Supprimer ces lignes de y_train et de X_train 
y_train = y_train[~mask]
X_train = X_train[~mask]

In [ ]:
# Pipeline pour les variables numériques (ici, toutes)
pipeline_num = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, X_train.columns)]
        )

# Pipeline finale
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso(alpha=0.1))
])

pipeline.fit(X_train, y_train)

In [ ]:
# Accéder au modèle à l'intérieur du pipeline
lasso_model = pipeline['model']

# Récupérer le nom des colonnes après transformation (surtout pour le OneHot)
feature_names = pipeline['preprocessor'].get_feature_names_out()

# Affichage des coefficients non nuls
coeffs = pd.Series(lasso_model.coef_, index=feature_names)
print("Variables sélectionnées (coeff != 0) :")
print(coeffs[coeffs != 0])

In [ ]:
# On applique l'imputation (pour boucher les NaN avec la moyenne)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_train)

# On applique la standardisation (obligatoire pour le LASSO)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# On peut maintenant lancer lasso_path sans erreur
alphas_to_test = [0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0]

alphas_computed, coefs_lasso, _ = lasso_path(
    X_scaled, 
    y_train, 
    alphas=alphas_to_test
)

# On compte combien de coefficients sont différents de 0 pour chaque colonne (axis=0)
n_nonzero = np.sum(coefs_lasso != 0, axis=0)

# Affichage des résultats sous forme de dictionnaire ou DataFrame
import pandas as pd
resultats_selection = pd.DataFrame({
    'Alpha': alphas_computed,
    'Variables_Selectionnees': n_nonzero
})

print(resultats_selection)

In [ ]:
my_alphas = np.array([0.001,0.01,0.02,0.025,0.05,0.1,0.25,0.5,0.8,1.0])

lcv = (
  LassoCV(
    alphas=my_alphas,
    fit_intercept=False,
    random_state=0,
    cv=5
    ).fit(
      X_scaled, y_train
    )
)

print("alpha optimal :", lcv.alpha_)

In [ ]:
# Pipeline pour les variables numériques (ici, toutes)
pipeline_num = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, X_train.columns)]
        )

# Pipeline finale
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso(alpha=lcv.alpha_))
])

lasso_optimal = pipeline.fit(X_train,y_train)

features_selec2 = extract_features_selected(lasso_optimal, 'preprocessor')

In [ ]:
features_selec2